<a href="https://colab.research.google.com/github/kkarbasi/FaMiniGPT/blob/master/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from faminigpt.utils import clone_persian_poems_colab
from faminigpt.gpt_modules import Head, MultiHeadAttention, MLP, PoetryModel
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
text = clone_persian_poems_colab('hafez')

In [3]:
print(text[0:500])


  	
الا یا ایها الساقی ادر کاسا و ناولها
که عشق آسان نمود اول ولی افتاد مشکل ها
به بوی نافه ای کاخر صبا زان طره بگشاید
ز تاب جعد مشکینش چه خون افتاد در دل ها
مرا در منزل جانان چه امن عیش چون هر دم
جرس فریاد می دارد که بربندید محمل ها
به می سجاده رنگین کن گرت پیر مغان گوید
که سالک بی خبر نبود ز راه و رسم منزل ها
شب تاریک و بیم موج و گردابی چنین هایل
کجا دانند حال ما سبکباران ساحل ها
همه کارم ز خود کامی به بدنامی کشید آخر
نهان کی ماند آن رازی کز او سازند محفل ها
حضوری گر همی خواهی از او غایب مشو 


In [4]:
# Simple encoder and decoders
chars = list(set(text))
vocab_size = len(chars)
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda x: [stoi[ch] for ch in x]
decode = lambda x: ''.join([itos[i] for i in x])

In [6]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train = data[:n]
test = data[n:]
print(train.shape, test.shape)

torch.Size([275735]) torch.Size([30638])


In [7]:
torch.manual_seed(1331)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# model
batch_size = 16
block_size = 32
embedding_size = 64
multihead_size = 64
num_heads = 4
num_transformers = 4
dropout = 0.00

assert multihead_size % num_heads == 0
head_size = int(multihead_size/num_heads)


# training loop
lr=1e-3
eval_iters = 200
max_iters = 5000
eval_interval = 100


In [8]:


def get_batch(split):
  data = train if split == 'train' else test
  idx = torch.randint(0, len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in idx])
  y = torch.stack([data[i+1:i+block_size+1] for i in idx])
  x = x.to(device)
  y = y.to(device)
  return x, y

In [9]:
x, y = get_batch('train')
print(x.shape)
print(y.shape)

torch.Size([16, 32])
torch.Size([16, 32])


In [10]:

def test_head():
  x, _ = get_batch('train')
  B, T = x.shape
  model = Head(head_size, embedding_size, block_size, dropout)
  model = model.to(device)
  embedding = nn.Embedding(vocab_size, embedding_size, device=device)
  head_output = model(embedding(x))
  assert head_output.shape == (B, T, head_size)
test_head()


In [11]:
def test_multihead():
  x, _ = get_batch('train')
  B, T = x.shape
  model = MultiHeadAttention(num_heads, head_size, embedding_size, block_size, dropout)
  model = model.to(device)
  embedding = nn.Embedding(vocab_size, embedding_size, device=device)
  multihead_output = model(embedding(x))
  assert multihead_output.shape == (B, T, head_size*num_heads)
test_multihead()

In [12]:

def test_poetry_model():
  x, y = get_batch('train')
  B, T = x.shape
  model = PoetryModel(vocab_size,
                     num_heads,
                     head_size,
                     embedding_size,
                     multihead_size,
                     block_size,
                     num_transformers,
                     dropout,
                     device)
  model = model.to(device)

  # without y (for generation)
  model_out = model(x)
  assert model_out[0].shape == (B, T, vocab_size)

  # with y (for training)
  model_out = model(x, y)
  assert model_out[0].shape == (B*T, vocab_size)

test_poetry_model()

In [13]:
model = PoetryModel(vocab_size,
                     num_heads,
                     head_size,
                     embedding_size,
                     multihead_size,
                     block_size,
                     num_transformers,
                     dropout,
                     device)
model = model.to(device)

In [14]:
optimizer = torch.optim.AdamW(params=model.parameters(), lr=lr)

In [15]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'test']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [16]:
for step in range(max_iters):
  if step % eval_interval == 0:
    eval_loss = estimate_loss()
    print(f"At step {step} train loss is {eval_loss['train']} test loss is {eval_loss['test']}")
  xb, yb = get_batch('train')
  logits, loss = model(xb, yb)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

At step 0 train loss is 3.84089732170105 test loss is 3.8649797439575195
At step 100 train loss is 2.6457202434539795 test loss is 2.7931811809539795
At step 200 train loss is 2.5704538822174072 test loss is 2.7577590942382812
At step 300 train loss is 2.5173888206481934 test loss is 2.729361057281494
At step 400 train loss is 2.477102279663086 test loss is 2.70937442779541
At step 500 train loss is 2.4502675533294678 test loss is 2.676898241043091
At step 600 train loss is 2.4188530445098877 test loss is 2.6389105319976807
At step 700 train loss is 2.3975510597229004 test loss is 2.625314235687256
At step 800 train loss is 2.368927240371704 test loss is 2.6086440086364746
At step 900 train loss is 2.339495897293091 test loss is 2.5896494388580322
At step 1000 train loss is 2.3310859203338623 test loss is 2.6080329418182373
At step 1100 train loss is 2.3045074939727783 test loss is 2.56663179397583
At step 1200 train loss is 2.290968656539917 test loss is 2.5552797317504883
At step 130

In [17]:
idx = torch.zeros((1, 1), dtype=torch.long, device=device) + stoi['\n']
print(decode(model.generate(idx, 2000)[0].tolist()))


بادگور از و شکو بار ای پیامی به چابید مسکل
با برفت کند کید بودی را همین بخود کدیر
او صبا باندو جام ابرود کای فحران
چو چمن چه راه خاکه و درا پرو داونی
به چو هر نیزی و خلب که عارصون دارم
تا چینید دهدی روران جای که عوکبی
چمحلخو چشمه اس جلاده تواننی
گشه گشه و زکر ز جامی سعر ملاصور بتازان بیاد
کین صحمی العش چه عود جهانم
و خبا جهف منکله کوچون فراه نفاد
گو رسوه پرده نهح ز باریم خنگی به تو کند
چه دست امود در چو من گشوید گران نمیدی
مگر حامعر بیماری رنخشان ذول چن پی
که دادار گشه حکایم زین
اختم می رباز بودیش سر که بیند خدم نکن
دنوشید ندید روی لبام که جامینا طلب
آشقهز بردش خدا ار که دورشیست به آنکن و نظوستی
به رود ره که تا ببگا آیید گو روی
چاه نویان دشی کو وجای اندان است
گو گوی ولامت و به نما بی همهی مجان هزار من
چو نقد اهر این و مدای دهانه قاک برافاز
به بیشماور به در کین منزان نمی کند
حدیی تروارانی که آید سیره خاک خونی
پرده وقران دهر می گوشه ما همیم
که گعندد در غبا پیمام دل دل از لدود
را رفوشفه جامرت
چو ترو به در از تو نیمه سخن ز عشقاله پیرد
خون سربش امن ضلعت سخن بکرد
کندی زبوحی نوله سروز چندان 

In [18]:
idx = torch.tensor(encode('کاوه'), dtype=torch.long, device=device).unsqueeze(dim=0)
print(decode(model.generate(idx, 600)[0].tolist()))

کاوه کند
بی در هر آمده که خیر ای جمیان حافظ
گز مرغ دله دم می بسیند که می باشی نیدم
از دیار دو ای نوک ای تنتمابه غلب
هوای ببود نفرون من به تخور از اندوسان بازآید
و به به آن و فر نیم خود جود مدوشم
بود می خاک آن دویدی همان مهرا تطف
حکیف او لمحک بر کهند کرده تو کند
تیخ مقای مقد شکن آهیم کنم
به ز و رخم دست از باید مایمارونی
بیار نماند صبا به ما برود آرده خور
خیرین رغ و نماندحش به دوش
ناله معند چو بیدام گریید دیند خوش باد
شمعری غمی آن گفتمم شدم حسن
مصند آنمانه کشی شهریب تو می بود مبو بینن
بکنشی خوشم صاراحتمی می نایبم و را
آن ز اهر که ننیمار با تطریب خلوی
محره گو نزم را لبا روی هندان لطفه گرانید
مگر ز چن
